# RSA 암호화 및 복호화

RSA 암호를 구현해봅시다.

[출처] https://github.com/Amaterazu7/rsa-python/blob/master/rsa.py

## 1. GCD 함수 만들기
우리 최대공약수를 구하는 알고리즘이 있었죠? 공개키를 생성하기 위해서 RSA에서도 이 알고리즘을 사용해서 서로소임을 확인하는 것이 필요합니다.  
해당 최대공약수 (GCD)를 구현합시다. 뭐를 이용해? 유클리드 알고리즘을 이용해서요.

In [4]:
def gcd(a, b):
    while b != 0:
        a, b = b, a % b
    return a

## 2. 곱셈에 대한 역원 구현하기
RSA의 키를 생성할 때 곱셈에 대한 역원을 통해 키 쌍을 만들었습니다. 곱셈에 대한 역원을 코드로 구현합시다.

In [5]:
def multiplicative_inverse(e, phi):
    d = 0
    x1 = 0
    x2 = 1
    y1 = 1
    temp_phi = phi

    while e > 0:
        temp1 = temp_phi//e
        temp2 = temp_phi - temp1 * e
        temp_phi = e
        e = temp2

        x = x2 - temp1 * x1
        y = d - temp1 * y1

        x2 = x1
        x1 = x
        d = y1
        y1 = y

    if temp_phi == 1:
        return d + phi

## 3. 키 쌍 생성하기
이제 위의 역원을 구현한 함수를 사용해서, n = pq가 있을 때 공개키와 비밀키 쌍을 생성해보겠습니다.  
공개키는 랜덤함수에서 하나 빼올게요.

In [7]:
import random

def generate_key_pair(p, q):
    if not (is_prime(p) and is_prime(q)):
        raise ValueError('Both numbers must be prime.')
    elif p == q:
        raise ValueError('p and q cannot be equal')
    n = p * q

    phi = (p-1) * (q-1)

    e = random.randrange(1, phi)

    g = gcd(e, phi)   ## 공개키는 phi함수를 적용한 n과 서로소여야 역수가 존재하겠습니다.
    while g != 1:
        e = random.randrange(1, phi)
        g = gcd(e, phi)

    d = multiplicative_inverse(e, phi)  ## 공개키와 비밀키는 역수 관계죠?

    return ((e, n), (d, n))

## 4. RSA 암호화 및 복호화
이제 암호화와 복호화를 구현합니다.

In [8]:
def encrypt(pk, plaintext):
    key, n = pk
    cipher = [pow(ord(char), key, n) for char in plaintext]
    return cipher


def decrypt(pk, ciphertext):
    key, n = pk
    aux = [str(pow(char, key, n)) for char in ciphertext]
    plain = [chr(int(char2)) for char2 in aux]
    return ''.join(plain)

## 5. 사용
이제 끝났습니다. 위의 함수를 사용해봅시다.  
p와 q를 소수로 설정하면, 공개키와 개인키는 내부 랜덤함수에 의해 알아서 생성될 것입니다.  
우리는 암호화와 복호화만 확인해봅시다.

In [12]:
p = 53
q = 59
public, private = generate_key_pair(p, q)
print(public, private)

(213, 3127) (1685, 3127)


In [13]:
cipher = encrypt(public, "hello")
print(cipher)

[2406, 1168, 615, 615, 1588]


In [14]:
plain = decrypt(private, cipher)
print(plain)

hello
